# Дообучение Gemma-2-2b с помощью Hugging Face

In [1]:
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from datasets import Dataset
from dotenv import load_dotenv
from peft import LoraConfig, get_peft_model
from transformers import (AutoModelForCausalLM, AutoTokenizer, Trainer,
                          TrainingArguments)

In [2]:
# Загрузка переменных из .env файла
load_dotenv()
# Чтение токена
token = os.getenv("HUGGING_FACE_ACCESS_TOKEN")
# Устройство для тензорных вычислений и хранения модели в памяти.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
model_name = "google/gemma-2-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    token=token,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## LoRA

Количество обучаемых параметров до применения LoRA.

In [4]:
def count_params(model):
    def pretty_number(num):
        return "{:,}".format(num).replace(",", " ")

    all_params = sum(p.numel() for p in model.parameters())
    print("Параметров:", pretty_number(all_params))
    grad_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Обучаемых параметров:", pretty_number(grad_params))


count_params(model)

Параметров: 2 614 341 888
Обучаемых параметров: 2 614 341 888


Применим LoRA к модели.

In [5]:
# Настройки LoRA
lora_config = LoraConfig(
    r=6,  # Ранг LoRA
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

Количество обучаемых параметров после применения LoRA.

In [6]:
count_params(model)

Параметров: 2 615 539 968
Обучаемых параметров: 1 198 080


## Подготовка данных

In [7]:
df = pd.read_csv("../data/prepared/data.csv")
df["name_ru"] = df["name_ru"].fillna("None")
df["text"] = df["text"].str.replace("\\n", " ")
df = df.drop_duplicates(ignore_index=True)

df["input"] = df.apply(
    lambda row: f"Аddress: {row['address']}; Name: {row['name_ru']}; Rating: {row['rating']}; Rubrics: {row['rubrics']}.",
    axis=1,
)
df["output"] = df.apply(lambda row: f"Text: {row["text"]}", axis=1)

df = df.drop(["address", "name_ru", "rating", "rubrics", "text"], axis=1)

print("Пример запроса:", df["input"][0])
print("Пример ответа:", df["output"][0])

Пример запроса: Аddress: Екатеринбург, ул. Московская / ул. Волгоградская / ул. Печатников; Name: Московский квартал; Rating: 3; Rubrics: Жилой комплекс.
Пример ответа: Text: Московский квартал 2. Шумно : летом по ночам дикие гонки. Грязно : кругом стройки, невозможно открыть окна (16 этаж! ), вечно по району летает мусор. Детские площадки убогие, на большой площади однотипные конструкции. Очень дорогая коммуналка. Часто срабатывает пожарная сигнализация. Жильцы уже не реагируют. В это время, обычно около часа, не работают лифты. Из плюсов - отличная планировка квартир ( Московская 194 ), на мой взгляд. Ремонт от застройщика на 3-. Окна вообще жуть - вместо вентиляции. По соотношению цена/качество - 3.

